In [27]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

from src.webscraping import (
    activate_web_driver,
    scrape_to_dataframe,
    convert_columns,
    combine_home_visitor,  
    get_todays_matchups,
)

from src.data_processing import (
    process_games,
    add_TARGET,
)

from src.feature_engineering import (
    process_features,
)

from src.hopsworks_utils import (
    save_feature_names,
    convert_feature_names,
)

import json

from pathlib import Path  #for Windows/Linux compatibility
DATAPATH = Path(r'data')

**Load API keys**

In [28]:
from dotenv import load_dotenv

load_dotenv()

try:
    HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']
except:
    raise Exception('Set environment variable HOPSWORKS_API_KEY')

**Activate Webdriver**

In [29]:
# initiate a webdriver in selenium 
# since website data is dynamically generated

driver = activate_web_driver('firefox')

2022-12-11 15:38:12,148 INFO: ====== WebDriver manager ======
2022-12-11 15:38:12,502 INFO: Get LATEST geckodriver version for 107.0 firefox
2022-12-11 15:38:13,124 INFO: Driver [C:\Users\Chris\.wdm\drivers\geckodriver\win64\0.32\geckodriver.exe] found in cache


**Scrape New Completed Games and Format**

In [30]:

def get_new_games(driver)-> pd.DataFrame:

    # set search for yesterday's games
    DAYS = 1
    SEASON = "" #no season will cause website to default to current season, format is "2022-23"
    TODAY = datetime.now(timezone('EST')) #nba.com uses US Eastern Standard Time
    LASTWEEK = (TODAY - timedelta(days=DAYS))
    DATETO = TODAY.strftime("%m/%d/%y")
    DATEFROM = LASTWEEK.strftime("%m/%d/%y")


    df = scrape_to_dataframe(driver, Season=SEASON, DateFrom=DATEFROM, DateTo=DATETO)

    df = convert_columns(df)

    print(df.info())
    df = combine_home_visitor(df)

    return df

df_new = get_new_games(driver)

# get the SEASON of the last game in the database
# this will used when constructing rows for prediction
SEASON = df_new['SEASON'].max()

df_new




<class 'pandas.core.frame.DataFrame'>
Int64Index: 102 entries, 2 to 103
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   HOME            102 non-null    int64         
 1   GAME_DATE_EST   102 non-null    datetime64[ns]
 2   HOME_TEAM_WINS  102 non-null    int64         
 3   PTS             102 non-null    int64         
 4   FG_PCT          102 non-null    float64       
 5   FG3_PCT         102 non-null    float64       
 6   FT_PCT          102 non-null    float64       
 7   REB             102 non-null    int64         
 8   AST             102 non-null    int64         
 9   TEAM_ID         102 non-null    object        
 10  GAME_ID         102 non-null    object        
dtypes: datetime64[ns](1), float64(3), int64(5), object(2)
memory usage: 9.6+ KB
None


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2022-12-10,1,136,52.0,32.1,71.9,59,19,1610612751,22200388,133,55.3,47.7,75.0,30,30,1610612754,2022
1,2022-12-10,1,114,42.6,39.6,93.8,42,22,1610612746,22200389,107,43.5,47.1,89.5,50,24,1610612764,2022
2,2022-12-10,0,102,45.8,34.5,80.0,40,25,1610612760,22200390,110,47.1,37.5,69.2,48,25,1610612739,2022
3,2022-12-10,1,115,53.8,33.3,77.4,46,24,1610612759,22200387,111,48.1,38.2,71.4,32,26,1610612748,2022
4,2022-12-10,0,107,43.7,30.0,73.1,39,17,1610612738,22200392,123,51.1,33.3,80.0,53,26,1610612744,2022
5,2022-12-10,0,110,45.9,36.6,81.0,42,20,1610612762,22200393,115,54.7,30.8,81.3,39,28,1610612743,2022
6,2022-12-10,0,118,54.2,43.5,75.0,28,22,1610612750,22200394,124,51.8,38.2,89.3,45,20,1610612757,2022
7,2022-12-10,0,115,45.6,25.0,84.6,37,24,1610612742,22200391,144,63.5,55.9,70.8,44,33,1610612741,2022
8,2022-12-09,1,106,43.7,34.4,82.6,51,27,1610612749,22200386,105,46.6,32.5,41.7,43,21,1610612742,2022
9,2022-12-09,1,106,53.0,48.5,62.5,38,21,1610612758,22200381,95,43.0,33.3,77.3,36,20,1610612739,2022


**Retrieve todays games**

In [31]:
#retrieve list of teams playing today

# get today's games on NBA schedule
teams_list = get_todays_matchups(driver)

# example output:
# ['/team/1610612759/spurs/', '/team/1610612748/heat/',...

print(teams_list)

# create list of matchups by parsing out team ids from teams_list
# second team id is always the home team
team_count = len(teams_list) 
matchups = []
for i in range(0,team_count,2):
    visitor_id = teams_list[i].partition("team/")[2].partition("/")[0] #extract team id from text
    home_id = teams_list[i+1].partition("team/")[2].partition("/")[0]
    matchups.append([visitor_id, home_id])

matchups

['/team/1610612756/suns/', '/team/1610612740/pelicans/', '/team/1610612747/lakers/', '/team/1610612765/pistons/', '/team/1610612758/kings/', '/team/1610612752/knicks/', '/team/1610612761/raptors/', '/team/1610612753/magic/', '/team/1610612766/hornets/', '/team/1610612755/sixers/', '/team/1610612741/bulls/', '/team/1610612737/hawks/', '/team/1610612749/bucks/', '/team/1610612745/rockets/']


[['1610612756', '1610612740'],
 ['1610612747', '1610612765'],
 ['1610612758', '1610612752'],
 ['1610612761', '1610612753'],
 ['1610612766', '1610612755'],
 ['1610612741', '1610612737'],
 ['1610612749', '1610612745']]

**Close Webdriver**

In [32]:
driver.close() 

**Create Blank Rows**

In [33]:
# append today's matchups to the new games dataframe

# since we don't have access to official game ids, we will use the 20000001 + index as a game id
# 20000001 is used to insure that feature engineering will treat this as a regular season game
# this data is only used for prediction and is deleted after the prediction is made

for i, matchup in enumerate(matchups):
    game_details = {'HOME_TEAM_ID': matchup[1], 
                    'VISITOR_TEAM_ID': matchup[0], 
                    'GAME_DATE_EST': datetime.now(timezone('EST')).strftime("%Y-%m-%d"), 
                    'GAME_ID': 20000001+i,
                    'SEASON': SEASON,
                    } 
    game_details_df = pd.DataFrame(game_details, index=[i])
    # append to new games dataframe
    df_today = pd.concat([df_today, game_details_df], ignore_index = True)

#blank rows will be filled with 0 to prevent issues with feature engineering
df_today = df_today.fillna(0) 

df_today


,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON
0,2022-12-10 00:00:00,1.0,136.0,52.0,32.1,71.9,59.0,19.0,1610612751,22200388,133.0,55.3,47.7,75.0,30.0,30.0,1610612754,2022
1,2022-12-10 00:00:00,1.0,114.0,42.6,39.6,93.8,42.0,22.0,1610612746,22200389,107.0,43.5,47.1,89.5,50.0,24.0,1610612764,2022
2,2022-12-10 00:00:00,0.0,102.0,45.8,34.5,80.0,40.0,25.0,1610612760,22200390,110.0,47.1,37.5,69.2,48.0,25.0,1610612739,2022
3,2022-12-10 00:00:00,1.0,115.0,53.8,33.3,77.4,46.0,24.0,1610612759,22200387,111.0,48.1,38.2,71.4,32.0,26.0,1610612748,2022
4,2022-12-10 00:00:00,0.0,107.0,43.7,30.0,73.1,39.0,17.0,1610612738,22200392,123.0,51.1,33.3,80.0,53.0,26.0,1610612744,2022
5,2022-12-10 00:00:00,0.0,110.0,45.9,36.6,81.0,42.0,20.0,1610612762,22200393,115.0,54.7,30.8,81.3,39.0,28.0,1610612743,2022
6,2022-12-10 00:00:00,0.0,118.0,54.2,43.5,75.0,28.0,22.0,1610612750,22200394,124.0,51.8,38.2,89.3,45.0,20.0,1610612757,2022
7,2022-12-10 00:00:00,0.0,115.0,45.6,25.0,84.6,37.0,24.0,1610612742,22200391,144.0,63.5,55.9,70.8,44.0,33.0,1610612741,2022
8,2022-12-09 00:00:00,1.0,106.0,43.7,34.4,82.6,51.0,27.0,1610612749,22200386,105.0,46.6,32.5,41.7,43.0,21.0,1610612742,2022
9,2022-12-09 00:00:00,1.0,106.0,53.0,48.5,62.5,38.0,21.0,1610612758,22200381,95.0,43.0,33.3,77.3,36.0,20.0,1610612739,2022


**Access Feature Store**

In [34]:
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY)
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/3350


Connected. Call `.close()` to terminate connection gracefully.


**Access Feature Group**

In [35]:
rolling_stats_fg = fs.get_feature_group(
    name="rolling_stats",
    version=1,
)

**Query Old Data Needed for Feature Engineering of New Data**

To generate features like rolling averages for the new games, older data from previous games is needed since some of the rolling averages might extend back 15 or 20 games or so.

In [36]:
BASE_FEATURES = ['game_date_est',
 'game_id',
 'home_team_id',
 'visitor_team_id',
 'season',
 'pts_home',
 'fg_pct_home',
 'ft_pct_home',
 'fg3_pct_home',
 'ast_home',
 'reb_home',
 'pts_away',
 'fg_pct_away',
 'ft_pct_away',
 'fg3_pct_away',
 'ast_away',
 'reb_away',
 'home_team_wins',
]

ds_query = rolling_stats_fg.select(BASE_FEATURES)
df_old = ds_query.read()
df_old


2022-12-11 15:41:49,288 INFO: USE `nba_predictor_featurestore`
2022-12-11 15:41:49,660 INFO: SELECT `fg0`.`game_date_est` `game_date_est`, `fg0`.`game_id` `game_id`, `fg0`.`home_team_id` `home_team_id`, `fg0`.`visitor_team_id` `visitor_team_id`, `fg0`.`season` `season`, `fg0`.`pts_home` `pts_home`, `fg0`.`fg_pct_home` `fg_pct_home`, `fg0`.`ft_pct_home` `ft_pct_home`, `fg0`.`fg3_pct_home` `fg3_pct_home`, `fg0`.`ast_home` `ast_home`, `fg0`.`reb_home` `reb_home`, `fg0`.`pts_away` `pts_away`, `fg0`.`fg_pct_away` `fg_pct_away`, `fg0`.`ft_pct_away` `ft_pct_away`, `fg0`.`fg3_pct_away` `fg3_pct_away`, `fg0`.`ast_away` `ast_away`, `fg0`.`reb_away` `reb_away`, `fg0`.`home_team_wins` `home_team_wins`
FROM `nba_predictor_featurestore`.`rolling_stats_1` `fg0`


,game_date_est,game_id,home_team_id,visitor_team_id,season,pts_home,fg_pct_home,ft_pct_home,fg3_pct_home,ast_home,reb_home,pts_away,fg_pct_away,ft_pct_away,fg3_pct_away,ast_away,reb_away,home_team_wins
0,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
1,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
2,2005-11-30,20500210,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
3,2018-12-10,21800395,1610612749,1610612739,2018,108,0.437988,0.817871,0.416992,22,58,92,0.375000,0.666992,0.333008,24,46,1
4,2007-03-12,20600946,1610612756,1610612745,2006,103,0.500000,0.727051,0.600098,18,50,82,0.385986,0.722168,0.262939,13,36,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22592,2022-12-07,22200372,1610612744,1610612762,2022,123,45.906250,76.000000,33.312500,26,42,124,53.500000,81.000000,42.906250,26,40,0
22593,2022-12-05,22200352,1610612746,1610612766,2022,119,48.406250,66.687500,41.687500,30,47,117,48.000000,64.687500,35.312500,29,49,1
22594,2022-12-03,22200340,1610612753,1610612761,2022,108,48.687500,80.812500,39.312500,23,29,121,56.000000,72.000000,31.000000,31,43,0
22595,2022-12-03,22200339,1610612749,1610612766,2022,105,47.000000,78.875000,37.500000,24,47,96,39.093750,88.500000,20.796875,15,44,1


**Convert Feature Names back to original mixed case**

In [37]:
df_old = convert_feature_names(df_old)
df_old

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2017-12-08,21700374,1610612759,1610612738,2017,105,0.468994,0.875000,0.295898,16,46,102,0.458008,0.881836,0.289062,14,39,1
1,2013-03-01,21200874,1610612756,1610612737,2012,92,0.444092,0.833008,0.455078,16,38,87,0.425049,0.772949,0.347900,21,43,1
2,2005-11-30,20500210,1610612738,1610612755,2005,110,0.447998,0.784180,0.250000,24,59,103,0.408936,0.770996,0.308105,21,40,1
3,2018-12-10,21800395,1610612749,1610612739,2018,108,0.437988,0.817871,0.416992,22,58,92,0.375000,0.666992,0.333008,24,46,1
4,2007-03-12,20600946,1610612756,1610612745,2006,103,0.500000,0.727051,0.600098,18,50,82,0.385986,0.722168,0.262939,13,36,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22592,2022-12-07,22200372,1610612744,1610612762,2022,123,45.906250,76.000000,33.312500,26,42,124,53.500000,81.000000,42.906250,26,40,0
22593,2022-12-05,22200352,1610612746,1610612766,2022,119,48.406250,66.687500,41.687500,30,47,117,48.000000,64.687500,35.312500,29,49,1
22594,2022-12-03,22200340,1610612753,1610612761,2022,108,48.687500,80.812500,39.312500,23,29,121,56.000000,72.000000,31.000000,31,43,0
22595,2022-12-03,22200339,1610612749,1610612766,2022,105,47.000000,78.875000,37.500000,24,47,96,39.093750,88.500000,20.796875,15,44,1


**Update Yesterday's Matchup Predictions with New Final Results**

In [ ]:
# filter out games that are pending final results
# (these were the rows used for prediction yesterday)
# and then update these with the new results

df_combined = df_old[df_old['GAME_ID'] < 200000100].update(df_new)

df_combined


**Add Today's Matchups for Feature Engineering**

In [ ]:
df_combined = pd.concat([df_combined, df_today], ignore_index = True)
df_combined

**Data Processing**

In [39]:
df_combined = process_games(df_combined) 
df_combined = add_TARGET(df_combined)
df_combined

,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,PTS_away,FG_PCT_away,FG3_PCT_away,FT_PCT_away,REB_away,AST_away,VISITOR_TEAM_ID,SEASON,PLAYOFF,TARGET
0,2022-12-10 00:00:00,1.0,136.0,52.0000,32.100000,71.9000,59.0,19.0,1610612751,22200388,133.0,55.30000,47.700000,75.000,30.0,30.0,1610612754,2022,0,1.0
1,2022-12-10 00:00:00,1.0,114.0,42.6000,39.600000,93.8000,42.0,22.0,1610612746,22200389,107.0,43.50000,47.100000,89.500,50.0,24.0,1610612764,2022,0,1.0
2,2022-12-10 00:00:00,0.0,102.0,45.8000,34.500000,80.0000,40.0,25.0,1610612760,22200390,110.0,47.10000,37.500000,69.200,48.0,25.0,1610612739,2022,0,0.0
3,2022-12-10 00:00:00,1.0,115.0,53.8000,33.300000,77.4000,46.0,24.0,1610612759,22200387,111.0,48.10000,38.200000,71.400,32.0,26.0,1610612748,2022,0,1.0
4,2022-12-10 00:00:00,0.0,107.0,43.7000,30.000000,73.1000,39.0,17.0,1610612738,22200392,123.0,51.10000,33.300000,80.000,53.0,26.0,1610612744,2022,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22571,2022-12-03 00:00:00,1.0,116.0,48.1875,40.000000,77.3750,49.0,19.0,1610612757,22200343,111.0,45.18750,36.093750,78.625,44.0,20.0,1610612762,2022,0,1.0
22572,2022-12-03 00:00:00,0.0,101.0,44.0000,10.703125,90.0000,45.0,17.0,1610612745,22200342,120.0,52.90625,48.093750,50.000,35.0,34.0,1610612744,2022,0,0.0
22588,2022-12-03 00:00:00,1.0,121.0,43.0000,39.312500,78.6250,50.0,27.0,1610612742,22200337,100.0,45.50000,32.312500,71.375,45.0,18.0,1610612752,2022,0,1.0
22594,2022-12-03 00:00:00,0.0,108.0,48.6875,39.312500,80.8125,29.0,23.0,1610612753,22200340,121.0,56.00000,31.000000,72.000,43.0,31.0,1610612761,2022,0,0.0


**Feature Engineering**

In [40]:
# Feature engineering to add: 
    # rolling averages of key stats, 
    # win/lose streaks, 
    # home/away streaks, 
    # specific matchup (team X vs team Y) rolling averages and streaks

df_combined = process_features(df_combined)
df_combined


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,GAME_ID,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
0,2003-10-28,1.0,89,0.439941,0.350098,0.533203,39,25,1610612755,20300001,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2003-10-28,1.0,109,0.505859,0.350098,0.600098,46,32,1610612747,20300003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2003-10-28,1.0,83,0.425049,0.099976,0.769043,38,20,1610612759,20300002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2003-10-29,1.0,88,0.323975,0.160034,0.700195,55,24,1610612740,20300006,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2003-10-29,1.0,98,0.506836,0.312988,0.730957,40,28,1610612738,20300004,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22607,2022-12-11,0.0,0,0.000000,0.000000,0.000000,0,0,1610612753,20000004,...,0.806470,0.536165,-0.666667,0.571429,2.4,1.266667,5.000000,0.428571,1.1,0.933333
22608,2022-12-11,0.0,0,0.000000,0.000000,0.000000,0,0,1610612765,20000002,...,3.836255,2.571240,2.000000,0.285714,0.5,0.800000,1.000000,-2.285714,0.1,1.400000
22609,2022-12-11,0.0,0,0.000000,0.000000,0.000000,0,0,1610612737,20000006,...,-5.026746,-3.348641,-6.000000,-0.571429,-0.6,-0.666667,2.333333,2.571429,2.2,0.200000
22610,2022-12-11,0.0,0,0.000000,0.000000,0.000000,0,0,1610612745,20000007,...,-6.578076,-4.393896,-11.000000,-6.285714,-4.4,-2.400000,-2.666667,-4.285714,-5.7,-6.733333


**Insert New Data into Feature Group**

In [41]:

def test():
    # retrieve only new games from the combined dataframe now that feature engineering is complete

    # set index to GAME_ID
    df_combined = df_combined.set_index('GAME_ID')
    df_new = df_new.set_index('GAME_ID')
    
    # retrieve only new games
    df_new = df_combined.loc[df_new.index]

    # reset GAME_ID index back to column
    df_new = df_new.reset_index()

    # convert certain features back to int32 for Hopsworks compatibility
    df_new['GAME_ID'] = df_new['GAME_ID'].astype('int32')
    df_new['HOME_TEAM_WINS'] = df_new['HOME_TEAM_WINS'].astype('int32')
    df_new['TARGET'] = df_new['TARGET'].astype('int32')

    # save new games to Hopsworks feature group
    rolling_stats_fg.insert(df_new, write_options={"wait_for_job" : False})

    df_new 

rolling_stats_fg.insert(df_combined, write_options={"wait_for_job" : False})

Uploading Dataframe: 0.00% |          | Rows 0/58 | Elapsed Time: 00:00 | Remaining Time: ?

Launching offline feature group backfill job...
Backfill Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/3350/jobs/named/rolling_stats_1_offline_fg_backfill/executions


,GAME_ID,GAME_DATE_EST,HOME_TEAM_WINS,PTS_home,FG_PCT_home,FG3_PCT_home,FT_PCT_home,REB_home,AST_home,HOME_TEAM_ID,...,FG3_PCT_AVG_LAST_10_ALL_x_minus_y,FG3_PCT_AVG_LAST_15_ALL_x_minus_y,AST_AVG_LAST_3_ALL_x_minus_y,AST_AVG_LAST_7_ALL_x_minus_y,AST_AVG_LAST_10_ALL_x_minus_y,AST_AVG_LAST_15_ALL_x_minus_y,REB_AVG_LAST_3_ALL_x_minus_y,REB_AVG_LAST_7_ALL_x_minus_y,REB_AVG_LAST_10_ALL_x_minus_y,REB_AVG_LAST_15_ALL_x_minus_y
0,22200388,2022-12-10,1,136,52.00000,32.093750,71.87500,59,19,1610612751,...,-4.057959,-2.719857,3.000000,1.571429,1.0,-2.000000,1.333333,-1.285714,-3.5,-3.266667
1,22200389,2022-12-10,1,114,42.59375,39.593750,93.81250,42,22,1610612746,...,6.324658,4.210970,3.000000,-2.285714,-2.8,-2.266667,7.666667,5.428571,5.7,6.933333
2,22200390,2022-12-10,0,102,45.81250,34.500000,80.00000,40,25,1610612760,...,3.252612,2.144141,4.666667,1.714286,0.7,0.800000,6.666667,0.857143,3.2,4.533333
3,22200387,2022-12-10,1,115,53.81250,33.312500,77.37500,46,24,1610612759,...,-4.840991,-3.218392,1.333333,-0.714286,1.1,2.733333,0.666667,1.000000,1.0,0.400000
4,22200392,2022-12-10,0,107,43.68750,30.000000,73.12500,39,17,1610612738,...,-0.009497,0.012240,-4.000000,-3.142857,-1.3,-1.266667,8.666667,3.571429,3.1,3.533333
5,22200393,2022-12-10,0,110,45.90625,36.593750,81.00000,42,20,1610612762,...,0.248718,0.167944,-4.666667,-5.142857,-4.6,-4.466667,-1.000000,1.857143,2.7,1.133333
6,22200394,2022-12-10,0,118,54.18750,43.500000,75.00000,28,22,1610612750,...,-1.375391,-0.936060,-0.333333,3.428571,6.2,4.266667,2.333333,6.142857,6.0,2.066667
7,22200391,2022-12-10,0,115,45.59375,25.000000,84.62500,37,24,1610612742,...,8.738647,5.830371,1.666667,-2.000000,0.5,-1.666667,-2.333333,2.428571,1.1,-2.266667
8,22200386,2022-12-09,1,106,43.68750,34.406250,82.62500,51,27,1610612749,...,-2.242432,-1.521908,4.000000,8.142857,5.5,4.066667,5.333333,5.714286,8.2,7.133333
9,22200381,2022-12-09,1,106,53.00000,48.500000,62.50000,38,21,1610612758,...,4.718677,3.134196,6.333333,1.857143,1.7,1.333333,3.000000,-2.000000,-0.5,-0.266667
